In [8]:
%load_ext autoreload
%autoreload 2
%gui qt
%matplotlib qt5

import sys
import os

#import cupy as cp
import numpy as np
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Metadata module

In [2]:
from oyLabImaging import Metadata

### Define path

In [3]:
fpath = '/bigstore/Microscopy Core/Jen/3T3_mRubyloss_HSV_20210818/'
MD = Metadata(fpath)

loaded ND2 metadata from/bigstore/Microscopy Core/Jen/3T3_mRubyloss_HSV_20210818/metadata.pickle


### You can read images by attributes

In [4]:
stk = MD.stkread(Channel = MD.channels[1], Position=MD.posnames[1], Zindex=[0],register=True)

opening index 8206
Loaded 1 group of images.


In [5]:
MD.CalculateDriftCorrection(Channel='DeepBlue', GPU=True)

opening file img_channel000_position002_time000000059_z000.tif
Loaded Pos2 group of images.

calculating drift correction for position Pos2
calculated drift correction for position Pos2
opening file img_channel000_position000_time000000059_z000.tif
Loaded Pos0 group of images.

calculating drift correction for position Pos0
calculated drift correction for position Pos0
opening file img_channel000_position001_time000000059_z000.tif
Loaded Pos1 group of images.

calculating drift correction for position Pos1
calculated drift correction for position Pos1


### use napari to see images

In [5]:
import napari
viewer = napari.Viewer()

2021-09-09 15:59:52,960 [INFO] WRITING LOG OUTPUT TO /home/alo649/.cellpose/run.log


In [6]:
viewer.add_image(stk)

<Image layer 'stk' at 0x7f79282f7760>

### you can export whatever is on your napari canvas to a video

In [ ]:
from oyLabImaging.Processing.imvisutils import export_napari_to_mp4
export_napari_to_mp4(filename='file name')

### If you want to segment cells, you can use a function called test_segmentation_params to see how the algorythm performs with different parameters. Any parameter value that's changed from the default should be manually added to the call for segmentation of the full data below.

In [9]:
from oyLabImaging.Processing.improcutils import segmentation
print(segmentation.segmentation_types()) 
img = MD.stkread(Position=MD.Position[1],Channel=MD.Channel[0], frame=10)
#imgCyto = MD.stkread(Channel='2YFP', Position='31', frame=50)
b = segmentation.test_segmentation_params(img=img, segment_type='cellpose_nuclei')#cellpose_nuclei
b

['watershed', 'cellpose_nuclei', 'cellpose_cyto', 'cellpose_nuc_cyto']
opening file img_channel000_position000_time000000010_z000.tif
Loaded Pos0 group of images.

using _segment_nuclei_cellpose


calculating with new parameters
Done!


In [ ]:
b.input_dict

### make empty results object

In [5]:
from oyLabImaging.Processing import results
R = results(MD=MD)


loaded results from pickle file


In [6]:
R()

Results object for path to experiment in path: 
 /bigstore/Microscopy Core/Jen/3T3_mRubyloss_HSV_20210818/

Available channels are : DIC N2, Widefield Green, Widefield Red, Widefield Dual Cyan, Widefield Blue.

Positions already segmented are : 0

Available positions : 0, 1, 2, 3, 4, 5.

Available frames : 92.


In [7]:
#doing this **b.input_dict will use the parameters you found earlier with segmentation.test_segmentation_params
R.setPosLbls(MD=MD,Position=R.PosNames[0],NucChannel=R.channels[0],segment_type='cellpose_nuclei', threads=6, **b.input_dict)


Processing position 0


100%|███████████████████████████████████████████| 92/92 [00:42<00:00,  2.15it/s]



Finished loading and segmenting position 0
saved results


In [11]:
MD.channels

array(['DeepBlue', 'Green', 'Red'], dtype=object)

In [8]:
R.calculate_tracks(pos=R.PosNames[0], params=[(MD.channels[0],1),(MD.channels[2],1)], search_radius=30, maxStep=15)

linking frame 58
Finished connecting tracks
saved results


In [9]:
R.show_points(R.PosNames[0], Channel=MD.channels[0])

showing channel DeepBlue
2021-09-09 15:48:47,663 [INFO] WRITING LOG OUTPUT TO /home/alo649/.cellpose/run.log


In [20]:
R.show_tracks(R.PosNames[0], J=np.arange(50))

showing channel DeepBlue


## Open track explorer in napari and look at your data.

In [ ]:
R.track_explorer()

## Ways to get data out, either to csv or as a matrix

In [ ]:
R.property_matrix?

In [ ]:
R.prop_to_csv?

In [35]:
t0 = R.tracks(0)

In [37]:
i=20
plt.plot(t0(i).T, t0(i).mean(MD.channels[4]))
plt.show()

### you can see a movie of a single tracked cell in different channels

In [ ]:
i=37
t0(i).show_movie(['DeepBlue','Red', 'Green'], cmaps=['cyan', 'magenta', 'yellow'])

### Routine for making a video with added plotting of fluorescence trajectories


In [ ]:

filename = 'your filename'
import matplotlib.animation as animation
dt=30
ims=[]
fig = plt.figure(facecolor='k')
ax1 = fig.add_subplot(2,1,1,facecolor='k')
ax2 = fig.add_subplot(2,1,2,facecolor='k')


ax2.spines['bottom'].set_color('w')
ax2.spines['top'].set_color('w')
ax2.spines['left'].set_color('w')
ax2.spines['right'].set_color('w')
ax2.tick_params(axis='x', colors='w')
ax2.tick_params(axis='y', colors='w')
for j in range(viewer.dims.range[0][1].astype(int)+1):
    viewer.dims.current_step=[j,0,0]
    ax1.set_position([0, 0, 0.5, 1])
    im = ax1.imshow(viewer.window.qt_viewer.canvas.render(),animated=True)
    t1 = ax1.text(10, 1, "%02d:%02d h"%(int(j*dt/60),j*dt%60),color='w')
    ax2.set_position([0.55, 0.3, 0.35, 0.4])
    # add as many trajectories as you want
    line1 = ax2.plot(t0(i).T[:j]/2, stats.zscore(t0(i).mean('DeepBlue'))[:j],'c',animated=True)
    line2 = ax2.plot(t0(i).T[:j]/2, stats.zscore(t0(i).mean('Red'))[:j],'m',animated=True)
    line3 = ax2.plot(t0(i).T[:j]/2, stats.zscore(t0(i).mean('Green'))[:j],'y',animated=True)
    line4 = ax2.plot(t0(i).T[:j]/2, stats.zscore(t0(i).area)[:j],'r',animated=True)
    ax2.legend(['Nuclei', 'mCherry', 'Death', 'Nuc. Area'], bbox_to_anchor=(0.7, 1.1), fancybox=True, facecolor='w', framealpha=1)
    ax2.set_xlabel('Time (h)', color='w')
    ax2.set_ylabel('AU', color='w')
    ims.append([t1, im, line1[0],line2[0],line3[0],line4[0],])
    

ani=animation.ArtistAnimation(fig,ims, interval=50,blit=True,repeat_delay=1000)

writer = animation.FFMpegWriter(fps=8, bitrate=1800)

ani.save(filename, writer=writer)